In [1]:
import os 
import sys

os.chdir("..")

# Path to be appended
new_path = "/home/gop/Desktop/quai/Quai-Macro-Model"

# Append the path to sys.path
if new_path not in sys.path:  # Avoid duplicate entries
    sys.path.append(new_path)


from MSML.src.math_spec_mapping import (load_from_json, create_action_chains_graph, write_out_boundary_actions,
                      write_out_policies, write_out_mechanisms, load_svg_graphviz, write_basic_report_full,
                      write_action_chain_reports, write_spec_tree, create_parameter_impact_table,
                     write_entity_reports, write_wiring_report, write_entity_markdown_report,
                     write_all_markdown_reports)

from math_spec_mapping import (load_from_json)

from IPython.display import display, display_html
from copy import deepcopy

from Spec.src import math_spec_json
import numpy as np
from pprint import pprint
from simulation import state_base, params_base, vesting_schedule_translate, build_logistic_classifier


state_base["K Qi"] = 1
state_base["K Quai"] = 500
state_base["Population Mining Beta Vector"] = np.array([-2, .0424242341])

ms = load_from_json(deepcopy(math_spec_json))
msi = ms.build_implementation(params_base)


Placeholder has no control action options


## Source Code for All Used Components

In [2]:
keys = [x.name for x in ms.wiring['Mine Block Wiring'].components_full()]
msi.print_source_code_files(keys)

--------------------Block Reward Policy--------------------
def block_reward_policy_v1(state, params, spaces):
    space = deepcopy(spaces[0])
    space["Quai Reward Offered"] = [
        state["Metrics"]["Hash to Quai Metric"](state, params, [{"Hash": x}])
        for x in space["Block Difficulty"]
    ]
    space["Qi Reward Offered"] = [
        state["Metrics"]["Hash to Qi Metric"](state, params, [{"Hash": x}])
        for x in space["Block Difficulty"]
    ]

    return [space]





--------------------Update Historical Quai Hash Mechanism--------------------
def update_historical_quai_hash_mechanism(state, params, spaces):
    if spaces[0]:
        state["Historical Quai Hash"].append(spaces[0])





--------------------Update Block Difficulty Mechanism--------------------
def update_block_difficulty_mechanism(state, params, spaces):

    if "Difficulty" in params["State Update Skipping Parameter"]:
        return

    new = (
        state["Block Difficulty"] * (params["Difficult

In [3]:
params_base_i = deepcopy(params_base)
params_base_i["FP Mining Payment Policy"] = "Deterministic Mining Payment Policy"
msi = ms.build_implementation(params_base_i)

state, params = msi.prepare_state_and_params(state_base, params_base_i,
                                             state_preperation_functions=[vesting_schedule_translate, build_logistic_classifier],
                                             parameter_preperation_functions=[])

spaces = []
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()


spaces = msi.policies["Block Reward Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()

spaces = msi.policies["Mining Payment Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()


spaces = msi.wiring["Mezzanine Mining Wiring"](state, params, spaces)
pprint(spaces)
print()
print()
print()

spaces = msi.wiring["Mining Mechanisms"](state, params, spaces)

Placeholder has no control action options
Aggregate Hashpower:  250000000.0
Number of Blocks:  16
Block Difficulties:  [2000000000, 4000000000, 64000000000, 2000000000, 4000000000, 4000000000, 4000000000, 2000000000, 16000000000, 8000000000, 2000000000, 8000000000, 2000000000, 4000000000, 4000000000, 16000000000]
Locking Options:  {0.038461538461538464: 1, 0.25: 1.035, 0.5: 1.1, 1: 1.25}
Locking Times:  [0.5, 1, 0.5, 0.25, 0.038461538461538464, 0.038461538461538464, 0.25, 0.5, 1, 0.5, 0.25, 0.25, 0.5, 0.038461538461538464, 0.25, 0.25]
Space:  {'Aggregate Hashpower': 250000000.0, 'Blocks to Mine': [{'Difficulty': 2000000000}, {'Difficulty': 4000000000}, {'Difficulty': 64000000000}, {'Difficulty': 2000000000}, {'Difficulty': 4000000000}, {'Difficulty': 4000000000}, {'Difficulty': 4000000000}, {'Difficulty': 2000000000}, {'Difficulty': 16000000000}, {'Difficulty': 8000000000}, {'Difficulty': 2000000000}, {'Difficulty': 8000000000}, {'Difficulty': 2000000000}, {'Difficulty': 4000000000}, {

KeyError: 'Qi Taken'

# Hashpower Comparisons

The current dummy model multiplies the Qi price by 1000. We display a low, medium and high scenario below to illustrate when hashpower is not enough, just right and too high and how difficulty adjusts as well as what the spaces look like.

The target mining time is set to 2 seconds.

## Low Hashpower

You can see that a prime block is not mined in the first epoch leading to a decrease in difficulty and two total epochs.

In [ ]:
np.random.seed(0)
state_i = deepcopy(state_base)
state_i["Qi Price"] = 5
state, params = msi.prepare_state_and_params(state_i, params_base_i, state_preperation_functions=[vesting_schedule_translate], parameter_preperation_functions=[])
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, [])
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Block Reward Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Payment Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()

## Medium Hashpower

One mining epoch happens and within a reasonable time so no difficulty adjustment

In [ ]:
np.random.seed(0)
state_i = deepcopy(state_base)
state_i["Qi Price"] = 10
state, params = msi.prepare_state_and_params(state_i, params_base_i, state_preperation_functions=[vesting_schedule_translate], parameter_preperation_functions=[])
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, [])
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Block Reward Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Payment Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()

## High Hashpower

One mining epoch happens but too fast so difficulty is adjusted to be higher

In [ ]:
np.random.seed(0)
state_i = deepcopy(state_base)
state_i["Qi Price"] = 1000
state, params = msi.prepare_state_and_params(state_i, params_base_i, state_preperation_functions=[vesting_schedule_translate], parameter_preperation_functions=[])
spaces = msi.boundary_actions['Mine Block Boundary Action'](state, params, [])
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Block Reward Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()
spaces = msi.policies["Mining Payment Policy"](state, params, spaces)
pprint(spaces)
print()
print()
print()